In [4]:
import requests
from pprint import pprint
import pandas as pd

import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string

from tqdm.notebook import tqdm


# 경고 무시 모듈
import warnings

# 경고를 무시
warnings.filterwarnings(action='ignore')

# 경고 보이기  
#warnings.filterwarnings(action='default')

In [1]:
api_key = input('국가교통정보센터_API_Key : ')

국가교통정보센터_API_Key : 


In [5]:
headers = {'Content-Type': 'application/json; charset=utf-8'}
params = {'apiKey' : api_key,
          'type' : 'all',
          'drcType' : 'all',
          'minX' : 128.67453,
          'maxX' : 129.01827,
          'minY' : 35.14605,
          'maxY' : 35.39879,
          'getType' : 'json'}
response = requests.get('https://openapi.its.go.kr:9443/trafficInfo', 
                        params=params, 
                        headers=headers)

In [6]:
response

<Response [200]>

In [7]:
a = response.json()

In [9]:
# 응답 코드  0 : 성공,  1: 실패
a['header']['resultCode']

0

In [10]:
# 찾은 도로 수
a['body']['totalCount']

14041

In [1]:
# roadName : 도로명
# roadDrcType : 도로 방향 유형
# linkNo : 일련 번호
# linkId : 링크 ID
# startNodeId : 시작 노드 ID
# endNodeId : 종료 노드 ID
# speed : 통행 속도 (h/km)
# travelTime : 통행 시간(s)
# createdDate : 데이터 생성 일시(YYYYMMDDHH24MISS)

In [12]:
df = pd.DataFrame(a['body']['items'])

In [13]:
df

roadName roadDrcType linkNo      linkId startNodeId   endNodeId speed  \
0            엄광로                     1340009902  1340003601  1440006500    19   
1            엄광로                     1340010001  1440006500  1340003601    23   
2           백양대로                     1340021101  1340006800  1340006801    43   
3           백양대로                     1340021202  1340006801  1340006800    47   
4      백양대로320번길                     1340021300  1340006800  1340008000    16   
...          ...         ...    ...         ...         ...         ...   ...   
14034          -                     4180379301  4180125301  4180125201    29   
14035          -                     4180379401  4180125301  4180125401  72.5   
14036          -                     4180379501  4180125401  4180125301    66   
14037          -                     4180379601  4180125401  4180125501    77   
14038          -                     4180379701  4180125501  4180125401  75.5   

      travelTime     createdDate  
0          85.04  20230118132500  
1          73.57  20230118132500  
2          18.21  20230118132500  
3          16.64  20230118132500  
4             61  20230118132500  
...          ...             ...  
14034      28.58  20230118132500  
14035      13.97  20230118132500  
14036      15.07  20230118132500  
14037      286.1  20230118132500  
14038     292.66  20230118132500  

[14039 rows x 9 columns]

In [14]:
df.to_csv('./전처리/교통정보/2023_01_13_15_55_00.csv', 
          encoding='cp949', index=False)

In [2]:
'''
LINK_ID      링크ID
F_NODE       시작노드ID
T_NODE       종료노드ID
LANES        차로수
ROAD_RANK    도로등급
ROAD_TYPE    도로유형
ROAD_NO      도로번호
ROAD_NAME    도로명
ROAD_USE     
MULTI_LINK   중용구간여부
CONNECT      연결로코드
MAX_SPD      최고제한속도
REST_VEH     통과제한차량
REST_W       통과제한하중
REST_H       통과제한높이
LENGTH       길이
REMARK       비고
'''

'\nLINK_ID      링크ID\nF_NODE       시작노드ID\nT_NODE       종료노드ID\nLANES        차로수\nROAD_RANK    도로등급\nROAD_TYPE    도로유형\nROAD_NO      도로번호\nROAD_NAME    도로명\nROAD_USE     \nMULTI_LINK   중용구간여부\nCONNECT      연결로코드\nMAX_SPD      최고제한속도\nREST_VEH     통과제한차량\nREST_W       통과제한하중\nREST_H       통과제한높이\nLENGTH       길이\nREMARK       비고\n'

In [15]:
df = pd.read_csv('./전처리/교통정보/2023_01_13_15_55_00.csv', encoding='cp949')

In [16]:
df.drop(['roadDrcType', 'linkNo'], axis=1, inplace=True)

In [18]:
epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
epsg5179 = from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")

In [19]:
link_df = gpd.GeoDataFrame.from_file('./전처리/교통정보/경남_도로_5179/경남_도로_5179.shp', encoding='cp949')
link_df.crs = epsg5179

In [21]:
link_df.drop('REMARK', axis=1, inplace=True)

In [23]:
link_df = link_df.astype({'LINK_ID':'int64'})

In [24]:
merge_right = pd.merge(link_df, df, how='left', left_on='LINK_ID', right_on='linkId')
merge_right

LINK_ID      F_NODE      T_NODE  LANES ROAD_RANK ROAD_TYPE ROAD_NO  \
0      1310014101  1310004600  1310004601      2       104       000       -   
1      1310014102  1310004601  1310005000      2       104       000       -   
2      1310014202  1310004601  1310004600      2       104       000       -   
3      1310014301  1310004800  1310004803      1       104       000       -   
4      1310014302  1310004803  1310004900      1       104       000       -   
...           ...         ...         ...    ...       ...       ...     ...   
41876  1310013501  1310004500  1310004501      1       104       000       -   
41877  1310013502  1310004501  1310004700      1       104       000       -   
41878  1310013601  1310004700  1310004501      1       104       000       -   
41879  1310013602  1310004501  1310004500      1       104       000       -   
41880  1310014000  1310004800  1310004600      2       104       000       -   

      ROAD_NAME ROAD_USE MULTI_LINK  ... REST_H      LENGTH  \
0          꽃마을로        0          0  ...      0  221.593705   
1          꽃마을로        0          0  ...      0  406.983746   
2          꽃마을로        0          0  ...      0  231.128863   
3         대신공원로        0          0  ...      0  209.486342   
4         대신공원로        0          0  ...      0  144.353284   
...         ...      ...        ...  ...    ...         ...   
41876       망양로        0          0  ...      0  272.795323   
41877       망양로        0          0  ...      0  323.745951   
41878       망양로        0          0  ...      0  331.185912   
41879       망양로        0          0  ...      0  284.077082   
41880       대신로        0          0  ...      0  365.033565   

                                                geometry  roadName  linkId  \
0      LINESTRING (1137898.705 1681169.424, 1137877.0...       NaN     NaN   
1      LINESTRING (1137784.093 1681349.488, 1137772.8...       NaN     NaN   
2      LINESTRING (1137773.785 1681344.880, 1137776.8...       NaN     NaN   
3      LINESTRING (1138182.290 1681324.835, 1138192.6...       NaN     NaN   
4      LINESTRING (1138173.639 1681531.280, 1138168.4...       NaN     NaN   
...                                                  ...       ...     ...   
41876  LINESTRING (1138284.316 1681125.703, 1138315.5...       NaN     NaN   
41877  LINESTRING (1138400.412 1681367.098, 1138403.3...       NaN     NaN   
41878  LINESTRING (1138642.183 1681197.837, 1138611.6...       NaN     NaN   
41879  LINESTRING (1138400.968 1681386.635, 1138398.5...       NaN     NaN   
41880  LINESTRING (1138177.774 1681334.085, 1138123.4...       NaN     NaN   

       startNodeId endNodeId speed  travelTime  createdDate  
0              NaN       NaN   NaN         NaN          NaN  
1              NaN       NaN   NaN         NaN          NaN  
2              NaN       NaN   NaN         NaN          NaN  
3              NaN       NaN   NaN         NaN          NaN  
4              NaN       NaN   NaN         NaN          NaN  
...            ...       ...   ...         ...          ...  
41876          NaN       NaN   NaN         NaN          NaN  
41877          NaN       NaN   NaN         NaN          NaN  
41878          NaN       NaN   NaN         NaN          NaN  
41879          NaN       NaN   NaN         NaN          NaN  
41880          NaN       NaN   NaN         NaN          NaN  

[41881 rows x 24 columns]

In [25]:
idx = merge_right[merge_right['linkId'].isnull()].index

In [26]:
merge_right.drop(idx, axis=0, inplace=True)

In [27]:
merge_right.reset_index(drop=True, inplace=True)

In [29]:
merge_right.columns

Index(['LINK_ID', 'F_NODE', 'T_NODE', 'LANES', 'ROAD_RANK', 'ROAD_TYPE',
       'ROAD_NO', 'ROAD_NAME', 'ROAD_USE', 'MULTI_LINK', 'CONNECT', 'MAX_SPD',
       'REST_VEH', 'REST_W', 'REST_H', 'LENGTH', 'geometry', 'roadName',
       'linkId', 'startNodeId', 'endNodeId', 'speed', 'travelTime',
       'createdDate'],
      dtype='object')

In [33]:
# 내가만든 교통상황

merge_right['User_Traffic_jam'] = None

for n in tqdm(range(len(merge_right))):
    if merge_right['MAX_SPD'][n] / 3 > merge_right['speed'][n]:
        merge_right['User_Traffic_jam'][n] = '정체'
        
    elif merge_right['MAX_SPD'][n] / 3 * 2 > merge_right['speed'][n]:
        merge_right['User_Traffic_jam'][n] = '서행'
        
    else:
        merge_right['User_Traffic_jam'][n] = '월활'     

  0%|          | 0/14016 [00:00<?, ?it/s]

In [35]:
# 지능형교통체계 표준 노드,링크 구축기준
# https://www.law.go.kr/LSW/admRulInfoP.do?admRulSeq=2000000023797

# ROAD_RANK
# 101  고속국도
# 102  도시고속국도
# 103  일반국도
# 104  특별,광역시도
# 105  국가지원지방도
# 106  지방도
# 107  시군도
# 108  기타

In [42]:
# 정규 교통상황

# 일반도로  104, 105, 106, 107
# 일반국도  103 
# 도시고속  102
# 고속도로  101


# 정체
# 일반도로 15 미만
# 일반국도 20 미만
# 도시고속 30 미만
# 고속도로 40 미만

# 서행
# 일반도로 15 이상 ~ 30 미만
# 일반국도 20 이상 ~ 40 미만
# 도시고속 30 이상 ~ 60 미만
# 고속도로 40 이상 ~ 70 미만

# 원활
# 일반도로 30 이상
# 일반국도 40 이상
# 도시고속 60 이상
# 고속도로 70 이상

merge_right['R_Traffic_jam'] = None

for n in tqdm(range(len(merge_right))):
    
    # 일반도로
    if merge_right['ROAD_RANK'][n] in ['104', '105', '106', '107']:
    
        if 15 > merge_right['speed'][n]:
            merge_right['R_Traffic_jam'][n] = '정체'

        elif 30 > merge_right['speed'][n]:
            merge_right['R_Traffic_jam'][n] = '서행'
        else:
            merge_right['R_Traffic_jam'][n] = '월활'
            
            
    # 일반국도        
    elif merge_right['ROAD_RANK'][n] == '103':
    
        if 20 > merge_right['speed'][n]:
            merge_right['R_Traffic_jam'][n] = '정체'

        elif 40 > merge_right['speed'][n]:
            merge_right['R_Traffic_jam'][n] = '서행'
        else:
            merge_right['R_Traffic_jam'][n] = '월활'  
            
            
    # 도시고속        
    elif merge_right['ROAD_RANK'][n] == '102':
    
        if 30 > merge_right['speed'][n]:
            merge_right['R_Traffic_jam'][n] = '정체'

        elif 60 > merge_right['speed'][n]:
            merge_right['R_Traffic_jam'][n] = '서행'
        else:
            merge_right['R_Traffic_jam'][n] = '월활'   
            
            
    # 고속도로        
    elif merge_right['ROAD_RANK'][n] == '101':
    
        if 40 > merge_right['speed'][n]:
            merge_right['R_Traffic_jam'][n] = '정체'

        elif 70 > merge_right['speed'][n]:
            merge_right['R_Traffic_jam'][n] = '서행'
        else:
            merge_right['R_Traffic_jam'][n] = '월활'   

  0%|          | 0/14016 [00:00<?, ?it/s]

In [39]:
merge_right['ROAD_RANK'].value_counts()

107    10461
104     1660
103      635
106      517
101      473
105      254
102       16
Name: ROAD_RANK, dtype: int64

In [43]:
merge_right

LINK_ID      F_NODE      T_NODE  LANES ROAD_RANK ROAD_TYPE ROAD_NO  \
0      3850050800  3850023800  3850023900      1       105       003      69   
1      3850112900  3850031400  3850031000      1       107       000       -   
2      1411072201  1410288507  1410404401      2       107       000       -   
3      3850113100  3850031300  3850033400      1       107       000       -   
4      3850113200  3850138400  3850138500      2       107       000       -   
...           ...         ...         ...    ...       ...       ...     ...   
14011  3880127100  3880045900  3880045500      1       107       000       -   
14012  3880127200  3880045500  3880045900      1       107       000       -   
14013  3880132600  3880047300  3880005100      2       106       002    1022   
14014  3880132700  3880005100  3880047300      2       106       002    1022   
14015  1411091001  1410409701  1410409801      1       107       000       -   

        ROAD_NAME ROAD_USE MULTI_LINK  ...  \
0             동북로        0          0  ...   
1          덕정로4번길        0          0  ...   
2      과학산단2로20번길        0          0  ...   
3         덕정로40번길        0          0  ...   
4        인제로284번길        0          0  ...   
...           ...      ...        ...  ...   
14011         금오로        0          0  ...   
14012         금오로        0          0  ...   
14013         황산로        0          0  ...   
14014         황산로        0          0  ...   
14015         정관길        0          0  ...   

                                                geometry    roadName  \
0      LINESTRING (1133447.794 1703292.291, 1133422.2...         동북로   
1      LINESTRING (1117495.865 1687612.552, 1117496.8...      덕정로4번길   
2      LINESTRING (1121406.951 1684759.202, 1121286.0...  과학산단2로20번길   
3      LINESTRING (1117436.520 1687611.599, 1117435.5...     덕정로40번길   
4      LINESTRING (1127581.265 1696242.348, 1127623.6...    인제로284번길   
...                                                  ...         ...   
14011  LINESTRING (1137737.055 1704053.172, 1137925.6...         금오로   
14012  LINESTRING (1137939.860 1703873.267, 1137751.3...         금오로   
14013  LINESTRING (1136968.601 1704958.725, 1137314.0...         황산로   
14014  LINESTRING (1137483.952 1705530.842, 1137421.5...         황산로   
14015  LINESTRING (1129931.412 1686272.244, 1129916.6...         정관길   

             linkId   startNodeId     endNodeId  speed travelTime  \
0      3.850051e+09  3.850024e+09  3.850024e+09    4.0      65.07   
1      3.850113e+09  3.850031e+09  3.850031e+09   20.0       7.43   
2      1.411072e+09  1.410289e+09  1.410404e+09   25.0      17.41   
3      3.850113e+09  3.850031e+09  3.850033e+09   15.0      13.92   
4      3.850113e+09  3.850138e+09  3.850138e+09   34.0       4.50   
...             ...           ...           ...    ...        ...   
14011  3.880127e+09  3.880046e+09  3.880046e+09    9.0     110.00   
14012  3.880127e+09  3.880046e+09  3.880046e+09   33.0      30.00   
14013  3.880133e+09  3.880047e+09  3.880005e+09   49.0      56.75   
14014  3.880133e+09  3.880005e+09  3.880047e+09   54.5      51.04   
14015  1.411091e+09  1.410410e+09  1.410410e+09   26.0      12.41   

        createdDate  User_Traffic_jam  R_Traffic_jam  
0      2.023012e+13                정체             정체  
1      2.023012e+13                서행             서행  
2      2.023012e+13                서행             서행  
3      2.023012e+13                정체             서행  
4      2.023012e+13                월활             월활  
...             ...               ...            ...  
14011  2.023012e+13                정체             정체  
14012  2.023012e+13                서행             월활  
14013  2.023012e+13                월활             월활  
14014  2.023012e+13                월활             월활  
14015  2.023012e+13                서행             서행  

[14016 rows x 26 columns]

In [44]:
merge_right.to_file('./전처리/교통정보/김해시_도로.shp', encoding='cp949', driver='ESRI Shapefile')